In [1]:
import os
os.chdir('../../')

In [2]:
from src.siamese import *
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image

In [3]:
pedia_model, ocr_model, logo_feat_list, file_name_list = phishpedia_config_OCR(num_classes=277,
                                                weights_path='./src/OCR/output/targetlist_lr0.01/bit.pth.tar',
                                                ocr_weights_path='./src/OCR/demo.pth.tar',
                                                targetlist_path='./src/phishpedia/expand_targetlist/')


using cuda.


  0%|          | 0/277 [00:00<?, ?it/s]/home/l/liny/anaconda3/envs/automl/lib/python3.7/site-packages/torch/nn/functional.py:3385: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior has changed "
  1%|▏         | 4/277 [00:06<06:43,  1.48s/it]/home/l/liny/anaconda3/envs/automl/lib/python3.7/site-packages/PIL/Image.py:963: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
100%|██████████| 277/277 [06:57<00:00,  1.51s/it]


In [4]:
logo_feat_list.shape

(3053, 2560)

In [7]:

domain_map_path = 'src/phishpedia/domain_map.pkl'
ct_benign = 0
ct_phish = 0


# for path in tqdm(os.listdir('/home/l/liny/ruofan/phishpedia/benchmark/Sampled_phish1000/')):
for path in tqdm(os.listdir('/home/l/liny/ruofan/phishpedia/benchmark/Sample_benign1000/')):
        
    url = '' # dummy value, not important
#     img_path = '/home/l/liny/ruofan/phishpedia/benchmark/Sampled_phish1000/' + path + '/shot.png'
#     annot = [x.strip().split(',') for x in open('/home/l/liny/ruofan/phishpedia/benchmark/phish1000_coord.txt').readlines()]
    img_path = '/home/l/liny/ruofan/phishpedia/benchmark/Sample_benign1000/' + path + '/shot.png'
    annot = [x.strip().split(',') for x in open('/home/l/liny/ruofan/phishpedia/benchmark/benign1000_coord.txt').readlines()]


    # read labelled 
    for c in annot:
        if c[0] == path:
            x1, y1, x2, y2 = map(float, c[1:])
            break
    pred_boxes = np.asarray([[x1, y1, x2, y2]])
    pred_classes = np.asarray([0.])
    
    # get predicted targeted brand
    pred_target, _, _ = phishpedia_classifier_OCR(pred_classes=pred_classes, pred_boxes=pred_boxes, 
                                                domain_map_path=domain_map_path,
                                                model=pedia_model, 
                                                ocr_model=ocr_model,
                                                logo_feat_list=logo_feat_list, 
                                                file_name_list=file_name_list,
                                                url=url,
                                                shot_path=img_path,
                                                ts=0.83)

    if pred_target is not None:
        ct_benign += 1  # if test on benign, look at this
        if brand_converter(pred_target) == brand_converter(path.split('+')[0]):
            ct_phish += 1 # if test on phish, look at this

  0%|          | 0/1000 [00:00<?, ?it/s]/home/l/liny/anaconda3/envs/automl/lib/python3.7/site-packages/torch/nn/functional.py:3385: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior has changed "
  6%|▌         | 55/1000 [00:43<06:25,  2.45it/s]/home/l/liny/anaconda3/envs/automl/lib/python3.7/site-packages/PIL/Image.py:963: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
100%|██████████| 1000/1000 [08:02<00:00,  2.07it/s]


In [8]:
# print(ct_phish, '/', len(os.listdir('/home/l/liny/ruofan/phishpedia/benchmark/Sampled_phish1000/')))
print(ct_benign, '/', len(os.listdir('/home/l/liny/ruofan/phishpedia/benchmark/Sample_benign1000/')))

2 / 1000
